# Clustering

# Calls to libraries 

In [117]:
from __future__ import annotations
import logging
from dataclasses import dataclass, field
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.impute import KNNImputer
from pycountry import countries
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger(__name__)


# Configuration for clustering analysis
Defining parameters. 

In [118]:
@dataclass
class Config:
    resources_path: str
    master_data_path: str = ""
    output_dir: str = ""
    
    n_clusters: int = 5
    min_clusters: int = 3
    max_clusters: int = 8
    random_state: int = 42
    
    # Feature engineering
    use_production_for_clustering: bool = True
    use_reserves_for_clustering: bool = False
    use_reserves_for_dominance: bool = True
    
    normalize_by_gdp: bool = True
    
    # Mineral aggregation
    aggregate_minerals: bool = True
    
    # Economic features
    include_economic_complexity: bool = False
    include_resource_rents: bool = False
    include_governance: bool = False
    include_development: bool = False
    
    # ECI imputation
    impute_missing_eci: bool = True
    
    # Resource diversity threshold
    diversity_threshold_pct: float = 1.0
    
    # Filtering
    year_min: int = 1990
    year_max: int = 2024
    master_year: int = 2019
    
    # Thresholds
    dominance_threshold: float = 15.0
    min_share_display: float = 1.0
    
    # Resource categorization
    energy_resources: list[str] = field(default_factory=lambda: ["Oil", "Natural Gas", "Coal"])
    
    def __post_init__(self):
        if self.output_dir:
            Path(self.output_dir).mkdir(parents=True, exist_ok=True)

## Safety test for country codes matching

In [119]:
class CountryCodeMapper:
    """Map country names to ISO codes."""
    
    def __init__(self):
        self.manual_mappings = {
            'US': 'USA',
            'UK': 'GBR',
            'Russia': 'RUS',
            'Venezuela': 'VEN',
            'Iran': 'IRN',
            'Syria': 'SYR',
            'Bolivia': 'BOL',
            'Vietnam': 'VNM',
            'Tanzania': 'TZA',
            'Brunei': 'BRN',
            'Congo': 'COG',
            'Ivory Coast': 'CIV',
            'Trinidad & Tobago': 'TTO',
            'South Korea': 'KOR',
            'North Korea': 'PRK',
            'Czech Republic': 'CZE',
            'Slovakia': 'SVK',
            'Czechia': 'CZE',
            'Laos': 'LAO',
            'Moldova': 'MDA',
            'Democratic Republic of Congo': 'COD',
            'DR Congo': 'COD',
            'Myanmar': 'MMR',
            'Niger': 'NER',  
            'Nigeria': 'NGA',
            'Kosovo': 'XKX', 
            'Serbia': 'SRB',
            'United States': 'USA', 
        }
    
    def get_code(self, country_name: str) -> str | None:
        """Get ISO3 code for a country name."""
        
        # Check manual mappings first
        if country_name in self.manual_mappings:
            return self.manual_mappings[country_name]
        
        # Try pycountry
        try:
            country = countries.search_fuzzy(country_name)
            if country:
                return country[0].alpha_3
        except LookupError:
            pass
        
        return None
    

## Data loader

In [120]:
class DataLoader:
    
    def __init__(self, config: Config):
        self.config = config
        self.mapper = CountryCodeMapper()
    
    def load_resources(self) -> pd.DataFrame:
        logger.info(f"Loading resources...")
        
        df = pd.read_csv(self.config.resources_path)
        
        if 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed: 0', axis=1)
        
        # Filter years
        df = df[(df['Year'] >= self.config.year_min) & (df['Year'] <= self.config.year_max)]
        
        # Add Country Code - OPTIMIZED: only map unique countries
        logger.info("  Mapping country names to ISO codes...")
        unique_countries = df['Country'].unique()
        country_mapping = {country: self.mapper.get_code(country) for country in unique_countries}
        df['Country Code'] = df['Country'].map(country_mapping)
        
        # Remove regional aggregates
        regional_keywords = ['Total', 'Other', 'Rest of']
        df = df[~df['Country'].str.contains('|'.join(regional_keywords), case=False, na=False)]
        
        # Remove rows without country codes
        before = len(df)
        df = df[df['Country Code'].notna()]
        after = len(df)
        
        if before > after:
            logger.info(f"  Removed {before - after} rows without country codes")
        
        logger.info(f"  {len(df):,} rows, {df['Resource'].nunique()} resources, {df['Country Code'].nunique()} countries")
        
        return df
    
    def load_master_data(self) -> pd.DataFrame:
        if not self.config.master_data_path:
            logger.info(f"No master data provided - will use only resource data")
            return pd.DataFrame()
            
        logger.info(f"Loading master data (wide format)...")
        
        df = pd.read_csv(self.config.master_data_path)
        
        if 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed: 0', axis=1)
        
        year = self.config.master_year
        df = df[df['Year'] == year].copy()
        
        logger.info(f"  Year {year}: {len(df)} countries")
        
        if 'GDP per capita (constant prices, PPP)' in df.columns:
            n_gdp = df['GDP per capita (constant prices, PPP)'].notna().sum()
            logger.info(f"  GDP per capita: {n_gdp} countries")
        
        if 'Economic Complexity Index' in df.columns:
            n_eci = df['Economic Complexity Index'].notna().sum()
            logger.info(f"  Economic Complexity: {n_eci} countries")
            
        return df
        

# Processing data

In [121]:
class DataProcessor:
    
    def __init__(self, config: Config):
        self.config = config
    
    def get_latest_values(self, df: pd.DataFrame) -> pd.DataFrame:
        """Get the most recent values for each country-resource-metric combination."""
        
        # Group by Country Code only (not Country name) to handle duplicates
        df_latest = (
            df.sort_values('Year', ascending=False)
            .groupby(['Country Code', 'Resource', 'Metric'])  # Remove 'Country' from groupby
            .agg({
                'Country': 'first',  # Take first country name
                'Value': 'first',     # Take most recent value
                'Year': 'first'
            })
            .reset_index()
        )
        
        return df_latest
    
    def calculate_production_values(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calculate production value (production × price), or use volume if no prices."""
        
        df_prod = df[df['Metric'] == 'Production'].copy()
        
        # Check if Price column exists in the dataframe columns
        has_price_column = 'Price' in df.columns
        
        if has_price_column:
            logger.info("    Calculating production values (production × price)...")
            df_price = df[df['Metric'] == 'Price'].copy()
            
            # Check price coverage
            resources_with_prod = set(df_prod['Resource'].unique())
            resources_with_price = set(df_price['Resource'].unique())
            missing_prices = resources_with_prod - resources_with_price
            
            if missing_prices:
                logger.info(f"      ⚠️  Resources without price data: {sorted(missing_prices)}")
                logger.info(f"      Using volume for resources without prices")
            
            # Merge production with prices
            df_prod = df_prod.merge(
                df_price[['Country Code', 'Resource', 'Year', 'Value']],
                on=['Country Code', 'Resource', 'Year'],
                how='left',
                suffixes=('_prod', '_price')
            )
            
            # Calculate value
            df_prod['Production_Value'] = df_prod['Value_prod'] * df_prod['Value_price']
            
            # Use volume as fallback where price is missing
            df_prod['Production_Value'] = df_prod['Production_Value'].fillna(df_prod['Value_prod'])
            
            n_with_price = df_prod['Value_price'].notna().sum()
            n_total = len(df_prod)
            logger.info(f"      {n_with_price}/{n_total} records have price data ({n_with_price/n_total*100:.1f}%)")
            
            df_prod = df_prod[['Country Code', 'Country', 'Resource', 'Production_Value', 'Year']]
            df_prod = df_prod.rename(columns={'Production_Value': 'Value'})
        else:
            logger.info("    No price data available - using production VOLUME")
            df_prod = df_prod[['Country Code', 'Country', 'Resource', 'Value', 'Year']]
        
        df_prod['Metric'] = 'Production'
        
        return df_prod
    
    def create_wide_format(self, df: pd.DataFrame) -> pd.DataFrame:
        """Convert to wide format with aggregation."""
        
        # Calculate production values (or use volume if no prices)
        df_prod = self.calculate_production_values(df)
        
        if self.config.aggregate_minerals:
            logger.info("    Aggregating minerals into one feature (keeping individual for hover)")
            
            df_energy = df_prod[df_prod['Resource'].isin(self.config.energy_resources)]
            df_minerals = df_prod[~df_prod['Resource'].isin(self.config.energy_resources)]
            
            # Pivot energy
            df_energy_wide = df_energy.pivot_table(
                index=['Country Code', 'Country'],
                columns='Resource',
                values='Value',
                aggfunc='first'
            ).reset_index()
            
            # Pivot individual minerals (for hover text - don't add _Prod suffix)
            df_minerals_individual = df_minerals.pivot_table(
                index=['Country Code', 'Country'],
                columns='Resource',
                values='Value',
                aggfunc='first'
            ).reset_index()
            
            # Rename individual minerals to avoid conflicts
            mineral_cols_rename = {c: f"{c}_Individual" for c in df_minerals_individual.columns 
                                  if c not in ['Country Code', 'Country']}
            df_minerals_individual = df_minerals_individual.rename(columns=mineral_cols_rename)
            
            # Sum minerals (for clustering)
            df_minerals_agg = df_minerals.groupby(['Country Code', 'Country'])['Value'].sum().reset_index()
            df_minerals_agg = df_minerals_agg.rename(columns={'Value': 'Minerals'})
            
            # Merge: energy + minerals aggregate + individual minerals
            df_prod_wide = df_energy_wide.merge(df_minerals_agg, on=['Country Code', 'Country'], how='outer')
            df_prod_wide = df_prod_wide.merge(df_minerals_individual, on=['Country Code', 'Country'], how='outer')
            
            # Rename only energy and aggregate for clustering
            prod_cols = {c: f"{c}_Prod" for c in df_prod_wide.columns 
                        if c not in ['Country Code', 'Country'] and c in list(self.config.energy_resources) + ['Minerals']}
        else:
            logger.info("    Keeping all resources separate")
            df_prod_wide = df_prod.pivot_table(
                index=['Country Code', 'Country'],
                columns='Resource',
                values='Value',
                aggfunc='first'
            ).reset_index()
            
            prod_cols = {c: f"{c}_Prod" for c in df_prod_wide.columns 
                        if c not in ['Country Code', 'Country']}
        
        df_prod_wide = df_prod_wide.rename(columns=prod_cols)
        logger.info(f"    Production: {len(df_prod_wide)} countries, {len(prod_cols)} features for clustering")
        logger.info(f"    (Individual minerals kept for hover text)")
        
        # Add reserves if needed
        if self.config.use_reserves_for_dominance:
            df_res = df[df['Metric'] == 'Reserves'].pivot_table(
                index=['Country Code', 'Country'],
                columns='Resource',
                values='Value',
                aggfunc='first'
            ).reset_index()
            
            res_cols = {c: f"{c}_Res" for c in df_res.columns 
                       if c not in ['Country Code', 'Country']}
            df_res = df_res.rename(columns=res_cols)
            
            df_prod_wide = df_prod_wide.merge(df_res, on=['Country Code', 'Country'], how='outer')
            logger.info(f"    Reserves (dominance only): {len(res_cols)} resources")
        
        return df_prod_wide
    
    def calculate_shares(self, df: pd.DataFrame) -> tuple[pd.DataFrame, list[str], list[str]]:
        """Calculate global shares for production and reserves."""
        
        df = df.copy()
        prod_cols = [c for c in df.columns if c.endswith("_Prod")]
        individual_cols = [c for c in df.columns if c.endswith("_Individual")]
        res_cols = [c for c in df.columns if c.endswith("_Res")]
        
        prod_share_cols, res_share_cols = [], []
        
        # Calculate shares for clustering features
        for col in prod_cols:
            total = df[col].sum()
            if total > 0:
                share_col = f"{col}_Share"
                df[share_col] = (df[col] / total) * 100
                prod_share_cols.append(share_col)
        
        # Calculate shares for individual minerals (for hover)
        for col in individual_cols:
            total = df[col].sum()
            if total > 0:
                share_col = f"{col}_Share"
                df[share_col] = (df[col] / total) * 100
                # Don't add to prod_share_cols (not used for clustering)
        
        for col in res_cols:
            total = df[col].sum()
            if total > 0:
                share_col = f"{col}_Share"
                df[share_col] = (df[col] / total) * 100
                res_share_cols.append(share_col)
        
        return df, prod_share_cols, res_share_cols
    
    def create_concentration_metrics(self, df: pd.DataFrame, share_cols: list[str]) -> pd.DataFrame:
        """Create dominance flags based on reserve concentration."""
        
        df = df.copy()
        threshold = self.config.dominance_threshold
        res_share_cols = [c for c in share_cols if '_Res_Share' in c]
        
        if res_share_cols:
            df['Max_Reserve_Concentration'] = df[res_share_cols].max(axis=1)
            df['Dominant_Count'] = (df[res_share_cols] >= threshold).sum(axis=1)
            df['Is_Dominant'] = df['Dominant_Count'] > 0
            
            df['Dominant_Resources'] = df.apply(
                lambda row: [col.replace('_Res_Share', '') for col in res_share_cols 
                           if pd.notna(row.get(col, 0)) and row.get(col, 0) >= threshold],
                axis=1
            )
            
            logger.info(f"  Dominant countries (>{threshold}% reserves): {df['Is_Dominant'].sum()}")
        
        return df
    
    def impute_eci(self, df: pd.DataFrame) -> pd.DataFrame:
        """Impute missing ECI based on similar countries."""
        
        if 'Economic Complexity Index' not in df.columns:
            return df
        
        df = df.copy()
        
        missing_eci = df[df['Economic Complexity Index'].isna()]['Country Code'].tolist()
        has_eci = df['Economic Complexity Index'].notna().sum()
        
        logger.info(f"  ECI data: {has_eci}/{len(df)} countries")
        
        if len(missing_eci) > 0:
            logger.info(f"  Countries missing ECI ({len(missing_eci)})")
            
            if self.config.impute_missing_eci and len(missing_eci) < len(df):
                logger.info(f"  Imputing missing ECI using KNN (k=5)...")
                
                impute_vars = ['Economic Complexity Index']
                proxy_vars = []
                
                if 'GDP per capita (constant prices, PPP)' in df.columns:
                    proxy_vars.append('GDP per capita (constant prices, PPP)')
                if 'Total natural resources rents (% of GDP)' in df.columns:
                    proxy_vars.append('Total natural resources rents (% of GDP)')
                if 'Manufacturing' in df.columns:
                    proxy_vars.append('Manufacturing')
                if 'Industry' in df.columns:
                    proxy_vars.append('Industry')
                
                if proxy_vars:
                    df_impute = df[proxy_vars + impute_vars].copy()
                    imputer = KNNImputer(n_neighbors=5)
                    df_imputed = pd.DataFrame(
                        imputer.fit_transform(df_impute),
                        columns=df_impute.columns,
                        index=df_impute.index
                    )
                    df['Economic Complexity Index'] = df_imputed['Economic Complexity Index']
                    logger.info(f"  ✓ Imputed ECI for {len(missing_eci)} countries")
                else:
                    df['Economic Complexity Index'] = df['Economic Complexity Index'].fillna(0)
            else:
                df['Economic Complexity Index'] = df['Economic Complexity Index'].fillna(0)
        
        return df
    
    def create_clustering_features(
        self, 
        df: pd.DataFrame,
        df_master: pd.DataFrame
    ) -> tuple[pd.DataFrame, list[str]]:
        """Create features for clustering."""
        
        df = df.copy()
        features = []
        
        # Merge with master data (if available)
        if len(df_master) > 0:
            logger.info("  Merging with master data...")
            
            countries_before = set(df['Country Code'].unique())
            df = df.merge(df_master, on='Country Code', how='left', suffixes=('', '_master'))
            countries_after = set(df['Country Code'].unique())
            master_countries = set(df_master['Country Code'].unique())
            
            overlap = countries_before & master_countries
            logger.info(f"  Resource data countries: {len(countries_before)}")
            logger.info(f"  Master data countries: {len(master_countries)}")
            logger.info(f"  Overlap: {len(overlap)}")
            
            if 'Country_master' in df.columns:
                df['Country'] = df['Country'].fillna(df['Country_master'])
                df = df.drop('Country_master', axis=1)
            
            if 'Year_master' in df.columns:
                df = df.drop('Year_master', axis=1)
            
            # Rename GDP column
            if 'GDP per capita (constant prices, PPP)' in df.columns:
                df = df.rename(columns={'GDP per capita (constant prices, PPP)': 'GDP_per_capita'})
            
            n_with_gdp = df['GDP_per_capita'].notna().sum() if 'GDP_per_capita' in df.columns else 0
            logger.info(f"  Matched {n_with_gdp} countries with GDP data")
            
            # Impute ECI if requested
            if self.config.include_economic_complexity:
                df = self.impute_eci(df)
        else:
            logger.info("  No master data - using only resource features")
        
        # Production features
        prod_cols = [c for c in df.columns if c.endswith("_Prod") and not c.endswith("_Share")]
        
        if self.config.normalize_by_gdp and 'GDP_per_capita' in df.columns:
            for col in prod_cols:
                feat = f"{col}_perGDPpc"
                df[feat] = np.where(
                    df['GDP_per_capita'].notna() & (df['GDP_per_capita'] > 0),
                    df[col].fillna(0) / (df["GDP_per_capita"] / 1000),
                    0
                )
                features.append(feat)
            
            agg = "aggregated" if self.config.aggregate_minerals else "separate"
            logger.info(f"  Created {len(prod_cols)} production VALUE/GDP features ({agg})")
        else:
            features.extend(prod_cols)
            logger.info(f"  Using {len(prod_cols)} production VALUE features (no GDP normalization)")
        
        # Resource diversity
        prod_share_cols = [c for c in df.columns if c.endswith("_Prod_Share")]
        threshold = self.config.diversity_threshold_pct
        
        df['Resource_Diversity'] = (df[prod_share_cols].fillna(0) > threshold).sum(axis=1)
        features.append('Resource_Diversity')
        logger.info(f"  Resource Diversity: Requires >{threshold}% global production")
        
        # Economic Complexity
        if self.config.include_economic_complexity and 'Economic Complexity Index' in df.columns:
            df['ECI'] = df['Economic Complexity Index'].fillna(0)
            features.append('ECI')
            logger.info(f"  Added ECI (optional)")
        
        logger.info(f"  Total clustering features: {len(features)}")
        
        return df, features

# Clustering engine + visualiser of data

In [122]:
# ============================================================================
# CLUSTERING ENGINE
# ============================================================================

class ClusteringEngine:
    
    def __init__(self, config: Config):
        self.config = config
        self.scaler = StandardScaler()
        self.kmeans = None
        self.pca = None
        self.loadings = None
        self.metrics = {}
    
    def _prepare_data(self, df: pd.DataFrame, features: list[str]) -> tuple[pd.DataFrame, np.ndarray]:
        
        df = df.copy()
        
        for feat in features:
            if feat in df.columns:
                df[feat] = df[feat].fillna(0)
            else:
                df[feat] = 0
        
        X = df[features].values.astype(np.float64)
        X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
        
        X_scaled = self.scaler.fit_transform(X)
        X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=0.0, neginf=0.0)
        
        return df, X_scaled
    
    def fit(self, df: pd.DataFrame, features: list[str]) -> pd.DataFrame:
        
        df, X_scaled = self._prepare_data(df, features)
        
        logger.info(f"  Clustering on {len(df)} countries with {len(features)} features")
        
        n_clusters = self.config.n_clusters
        self.kmeans = KMeans(
            n_clusters=n_clusters,
            random_state=self.config.random_state,
            n_init=10,
            max_iter=300
        )
        
        df["cluster"] = self.kmeans.fit_predict(X_scaled)
        
        self.metrics['silhouette'] = silhouette_score(X_scaled, df["cluster"])
        self.metrics['davies_bouldin'] = davies_bouldin_score(X_scaled, df["cluster"])
        self.metrics['inertia'] = self.kmeans.inertia_
        
        logger.info(f"  ✓ Silhouette: {self.metrics['silhouette']:.3f}, Davies-Bouldin: {self.metrics['davies_bouldin']:.3f}")
        logger.info(f"  Distribution: {df['cluster'].value_counts().sort_index().to_dict()}")
        
        # PCA
        self.pca = PCA(n_components=2)
        pca_result = self.pca.fit_transform(X_scaled)
        df["PC1"], df["PC2"] = pca_result[:, 0], pca_result[:, 1]
        
        variance = self.pca.explained_variance_ratio_
        logger.info(f"  PCA: PC1={variance[0]*100:.1f}%, PC2={variance[1]*100:.1f}%, Total={variance.sum()*100:.1f}%")
        
        self.loadings = pd.DataFrame(
            self.pca.components_.T,
            columns=["PC1", "PC2"],
            index=features
        )
        
        return df
    
    def find_optimal_k(self, df: pd.DataFrame, features: list[str]) -> dict:
        
        logger.info("Finding optimal k...")
        
        df_temp, X_scaled = self._prepare_data(df, features)
        
        k_range = range(self.config.min_clusters, self.config.max_clusters + 1)
        results = []
        
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=self.config.random_state, n_init=50)
            labels = kmeans.fit_predict(X_scaled)
            
            sil = silhouette_score(X_scaled, labels)
            db = davies_bouldin_score(X_scaled, labels)
            
            results.append({'k': k, 'silhouette': sil, 'davies_bouldin': db, 'inertia': kmeans.inertia_})
            logger.info(f"  k={k}: Silhouette={sil:.3f}, Davies-Bouldin={db:.3f}")
        
        df_results = pd.DataFrame(results)
        optimal_k = df_results.loc[df_results['silhouette'].idxmax(), 'k']
        logger.info(f"✓ Optimal k: {optimal_k}")
        
        return {'results': df_results, 'optimal_k': int(optimal_k)}

class Visualizer:
    
    def __init__(self, config: Config):
        self.config = config
        self.colors = px.colors.qualitative.Bold
    
    def clean_label(self, text: str) -> str:
        """Clean labels for visualization."""
        return (text.replace("_perGDPpc", "/GDP")
                   .replace("_Prod", " Prod")
                   .replace("_", " "))
    
    def create_hover_text(self, row: pd.Series, features: list[str], share_cols: list[str]) -> str:
        
        threshold = self.config.dominance_threshold
        min_share = self.config.min_share_display
        
        lines = [f"<b>{row['Country']}</b>", f"Cluster: {int(row['cluster'])}"]
        
        if row.get("Is_Dominant"):
            lines.append(f"🔴 >{threshold}% RESERVES")
            for resource in row.get("Dominant_Resources", []):
                share = row.get(f"{resource}_Res_Share", 0)
                if pd.notna(share):
                    lines.append(f"   • {resource}: {share:.1f}%")
        
        if pd.notna(row.get('GDP_per_capita')):
            lines.append(f"GDP/capita: ${row['GDP_per_capita']:,.0f}")
        if pd.notna(row.get('ECI')):
            lines.append(f"Econ Complexity: {row['ECI']:.2f}")
        if pd.notna(row.get('Resource_Diversity')):
            threshold_pct = self.config.diversity_threshold_pct
            lines.append(f"Resources (>{threshold_pct}% global): {int(row['Resource_Diversity'])}")
        
        # Energy resources (Oil, Gas, Coal) - show if significant
        energy_resources = ['Oil', 'Natural Gas', 'Coal']
        energy_shares = []
        
        for resource in energy_resources:
            share_col = f"{resource}_Prod_Share"
            if share_col in row.index:
                share = row.get(share_col, 0)
                if pd.notna(share) and share > min_share:
                    energy_shares.append((resource, share))
        
        if energy_shares:
            lines.append("")
            lines.append("⚡ Energy Resources:")
            for resource, val in energy_shares:
                lines.append(f"   {resource}: {val:.1f}%")
        
        # Disaggregated minerals - show top 3 by production share
        # Get all mineral shares (individual minerals with _Individual suffix)
        mineral_shares = []
        for col in row.index:
            if '_Individual_Share' in col:
                resource = col.replace('_Individual_Share', '')
                share = row.get(col, 0)
                if pd.notna(share) and share > min_share:
                    mineral_shares.append((resource, share))
        
        # Sort by share and take top 3
        mineral_shares.sort(key=lambda x: x[1], reverse=True)
        
        if mineral_shares:
            lines.append("")
            lines.append("⛏️ Top Minerals:")
            for resource, val in mineral_shares[:3]:
                lines.append(f"   {resource}: {val:.1f}%")
            
            # If there are more, indicate it
            if len(mineral_shares) > 3:
                lines.append(f"   ... +{len(mineral_shares) - 3} more")
        
        return "<br>".join(lines)
    
    def choropleth_map(self, df: pd.DataFrame, features: list[str], share_cols: list[str]) -> go.Figure:
        
        df_map = df[df["Country Code"].notna()].copy()
        n_clusters = df_map["cluster"].nunique()
        threshold = self.config.dominance_threshold
        
        df_map["hover_text"] = df_map.apply(
            lambda row: self.create_hover_text(row, features, share_cols), axis=1
        )
        
        fig = go.Figure()
        
        for cid in range(n_clusters):
            subset = df_map[(df_map["cluster"] == cid) & (~df_map["Is_Dominant"])]
            if len(subset) > 0:
                fig.add_trace(go.Choropleth(
                    locations=subset["Country Code"],
                    z=[cid] * len(subset),
                    colorscale=[[0, self.colors[cid]], [1, self.colors[cid]]],
                    showscale=False,
                    customdata=subset["hover_text"].values,
                    hovertemplate="%{customdata}<extra></extra>",
                    name=f"Cluster {cid} ({len(subset)})",
                    marker=dict(line=dict(color="white", width=0.5))
                ))
        
        for cid in range(n_clusters):
            subset = df_map[(df_map["cluster"] == cid) & (df_map["Is_Dominant"])]
            if len(subset) > 0:
                fig.add_trace(go.Choropleth(
                    locations=subset["Country Code"],
                    z=[cid] * len(subset),
                    colorscale=[[0, self.colors[cid]], [1, self.colors[cid]]],
                    showscale=False,
                    customdata=subset["hover_text"].values,
                    hovertemplate="%{customdata}<extra></extra>",
                    name=f"Cluster {cid} 🔴 ({len(subset)})",
                    marker=dict(line=dict(color="red", width=4))
                ))
        
        fig.update_geos(projection_type="natural earth", showcountries=True, showcoastlines=True)
        
        agg = " (Minerals Aggregated)" if self.config.aggregate_minerals else ""
        
        fig.update_layout(
            title=f"Resource Clustering by Production Value{agg} (k={n_clusters})<br>"
                  f"<sup>🔴 >{threshold}% reserves | Reserves NOT in clustering</sup>",
            width=1400, height=800,
            legend=dict(x=0.02, y=0.5, bgcolor="rgba(255,255,255,0.8)")
        )
        
        return fig
    
    def cluster_profiles(self, df: pd.DataFrame, features: list[str]) -> go.Figure:
        
        centroids = df.groupby("cluster")[features].mean()
        clean_labels = [self.clean_label(f) for f in features]
        
        centroid_z = pd.DataFrame(
            StandardScaler().fit_transform(centroids),
            index=[f"Cluster {i}" for i in centroids.index],
            columns=clean_labels
        )
        
        fig = go.Figure(data=go.Heatmap(
            z=centroid_z.values,
            x=centroid_z.columns,
            y=centroid_z.index,
            colorscale="RdBu_r",
            zmid=0,
            text=np.round(centroid_z.values, 2),
            texttemplate="%{text}",
            hovertemplate="<b>%{y}</b><br>%{x}<br>Z: %{z:.2f}<extra></extra>",
            colorbar=dict(title="Z-Score")
        ))
        
        fig.update_layout(
            title="Cluster Profiles (Z-Scores)",
            height=500, width=1400,
            xaxis=dict(tickangle=-45)
        )
        
        return fig
    
    def validation_plots(self, optimization_results: dict) -> go.Figure:
        
        df = optimization_results['results']
        optimal_k = optimization_results['optimal_k']
        
        fig = make_subplots(rows=1, cols=3, subplot_titles=('Silhouette', 'Davies-Bouldin', 'Inertia'))
        
        fig.add_trace(go.Scatter(x=df['k'], y=df['silhouette'], mode='lines+markers'), row=1, col=1)
        fig.add_vline(x=optimal_k, line_dash="dash", line_color="red", row=1, col=1)
        
        fig.add_trace(go.Scatter(x=df['k'], y=df['davies_bouldin'], mode='lines+markers',
                                line=dict(color='orange')), row=1, col=2)
        
        fig.add_trace(go.Scatter(x=df['k'], y=df['inertia'], mode='lines+markers',
                                line=dict(color='green')), row=1, col=3)
        
        fig.update_layout(title_text=f"Validation (Optimal k={optimal_k})", height=400, width=1400, showlegend=False)
        
        return fig

# Display results of clustering and run the main function

In [123]:
class ResourceClusteringAnalysis:
    
    def __init__(self, config: Config):
        self.config = config
        self.loader = DataLoader(config)
        self.processor = DataProcessor(config)
        self.clusterer = ClusteringEngine(config)
        self.visualizer = Visualizer(config)
        
        self.df_final = None
        self.clustering_features = None
        self.prod_share_cols = None
        self.res_share_cols = None
        self.optimization_results = None
    
    def _display_data_availability(self, df: pd.DataFrame):
        """Display comprehensive data availability by resource, metric, and year."""
        
        logger.info("\n" + "="*80)
        logger.info("DATA AVAILABILITY ANALYSIS")
        logger.info("="*80)
        
        # Get production data
        prod_data = df[df['Metric'] == 'Production'].copy()
        
        # Create summary by resource
        logger.info("\nPRODUCTION DATA BY RESOURCE:")
        logger.info("-" * 80)
        
        for resource in sorted(prod_data['Resource'].unique()):
            resource_data = prod_data[prod_data['Resource'] == resource]
            years = sorted(resource_data['Year'].unique())
            countries = resource_data['Country'].nunique()
            
            logger.info(f"\n{resource}:")
            logger.info(f"  Years: {min(years)} - {max(years)} ({len(years)} years)")
            logger.info(f"  Countries: {countries}")
            logger.info(f"  Total records: {len(resource_data):,}")
        
        # Reserves data
        res_data = df[df['Metric'] == 'Reserves'].copy()
        
        if len(res_data) > 0:
            logger.info("\n" + "-" * 80)
            logger.info("RESERVES DATA BY RESOURCE:")
            logger.info("-" * 80)
            
            for resource in sorted(res_data['Resource'].unique()):
                resource_data = res_data[res_data['Resource'] == resource]
                years = sorted(resource_data['Year'].unique())
                countries = resource_data['Country'].nunique()
                
                logger.info(f"\n{resource}:")
                logger.info(f"  Years: {min(years)} - {max(years)} ({len(years)} years)")
                logger.info(f"  Countries: {countries}")
                logger.info(f"  Total records: {len(resource_data):,}")
        
        # Year coverage summary
        logger.info("\n" + "="*80)
        logger.info("YEAR COVERAGE SUMMARY (Production)")
        logger.info("="*80)
        
        year_coverage = []
        all_years = sorted(prod_data['Year'].unique())
        
        # Sample key years
        key_years = [1990, 2000, 2010, 2014, 2015, 2020, 2023, 2024]
        display_years = [y for y in key_years if y in all_years]
        
        logger.info(f"\nResources by year ({', '.join(map(str, display_years))}):")
        logger.info("-" * 80)
        
        for resource in sorted(prod_data['Resource'].unique()):
            resource_data = prod_data[prod_data['Resource'] == resource]
            coverage = []
            for year in display_years:
                year_data = resource_data[resource_data['Year'] == year]
                countries = year_data['Country'].nunique() if len(year_data) > 0 else 0
                coverage.append(f"{countries:>3}")
            
            logger.info(f"{resource:<20} {' '.join(coverage)}")
        
        # Create availability visualization
        logger.info("\n" + "="*80)
        logger.info("Creating availability heatmap...")
        
        
        # Create matrix for heatmap
        availability_matrix = prod_data.groupby(['Resource', 'Year'])['Country'].nunique().reset_index()
        availability_pivot = availability_matrix.pivot(index='Resource', columns='Year', values='Country')
        availability_pivot = availability_pivot.fillna(0)
        
        # Sort by total coverage
        availability_pivot['_total'] = availability_pivot.sum(axis=1)
        availability_pivot = availability_pivot.sort_values('_total', ascending=False)
        availability_pivot = availability_pivot.drop('_total', axis=1)
        
        # Focus on 1990 onwards if available
        if 1990 in availability_pivot.columns:
            availability_pivot = availability_pivot.loc[:, 1990:]
        
        fig = go.Figure(data=go.Heatmap(
            z=availability_pivot.values,
            x=availability_pivot.columns,
            y=availability_pivot.index,
            colorscale='Viridis',
            text=availability_pivot.values.astype(int),
            texttemplate='%{text}',
            hovertemplate='<b>%{y}</b><br>Year: %{x}<br>Countries: %{z}<extra></extra>',
            colorbar=dict(title='Number of<br>Countries')
        ))
        
        fig.update_layout(
            title='Production Data Availability: Number of Countries by Resource and Year',
            xaxis_title='Year',
            yaxis_title='Resource',
            height=600,
            width=1400
        )
        
        fig.show()
        logger.info("✓ Availability heatmap displayed")
    
    def run(self, find_optimal_k: bool = False) -> pd.DataFrame:
        
        logger.info("="*80)
        logger.info("NATURAL RESOURCES CLUSTERING - PRODUCTION VALUES DATASET")
        logger.info("="*80)
        
        logger.info("\n[1] LOADING DATA")
        df_resources = self.loader.load_resources()
        df_master = self.loader.load_master_data()
        
        # Display data availability (VERY TIME CONSUMING)
        #self._display_data_availability(df_resources)
        
        logger.info("\n[2] PROCESSING")
        df_latest = self.processor.get_latest_values(df_resources)
        df_wide = self.processor.create_wide_format(df_latest)
        
        logger.info("\n[3] CALCULATING SHARES")
        df_wide, self.prod_share_cols, self.res_share_cols = self.processor.calculate_shares(df_wide)
        
        logger.info("\n[4] DOMINANCE FROM RESERVES")
        all_share_cols = self.prod_share_cols + self.res_share_cols
        df_wide = self.processor.create_concentration_metrics(df_wide, all_share_cols)
        
        logger.info("\n[5] CREATING CLUSTERING FEATURES")
        df_wide, self.clustering_features = self.processor.create_clustering_features(df_wide, df_master)
        
        if find_optimal_k:
            logger.info("\n[6] FINDING OPTIMAL K")
            self.optimization_results = self.clusterer.find_optimal_k(df_wide, self.clustering_features)
        
        logger.info(f"\n[{'7' if find_optimal_k else '6'}] CLUSTERING")
        self.df_final = self.clusterer.fit(df_wide, self.clustering_features)
        
        self._print_summary()
        
        return self.df_final
    
    def _print_summary(self):
        
        logger.info("\n" + "="*80)
        logger.info("CLUSTERS")
        logger.info("="*80)
        
        for c in range(self.df_final["cluster"].nunique()):
            cluster_df = self.df_final[self.df_final["cluster"] == c]
            
            logger.info(f"\nCluster {c} ({len(cluster_df)} countries):")
            countries = sorted(cluster_df["Country"].tolist())
            logger.info(f"  {', '.join(countries[:12])}{'...' if len(countries) > 12 else ''}")
            
            dominant = cluster_df[cluster_df["Is_Dominant"]]["Country"].tolist()
            if dominant:
                logger.info(f"  🔴 High reserves: {', '.join(dominant)}")
    
    def visualize(self) -> dict[str, go.Figure]:
        
        if self.df_final is None:
            raise RuntimeError("Run .run() first")
        
        all_share_cols = self.prod_share_cols + self.res_share_cols
        
        figures = {
            "map": self.visualizer.choropleth_map(self.df_final, self.clustering_features, all_share_cols),
            "profiles": self.visualizer.cluster_profiles(self.df_final, self.clustering_features),
        }
        
        if self.optimization_results:
            figures["validation"] = self.visualizer.validation_plots(self.optimization_results)
        
        return figures
    
    def show_figures(self):
        for name, fig in self.visualize().items():
            fig.show()
    
    def export_results(self, output_path: str):
        
        base_cols = ["Country Code", "Country", "cluster"]
        optional = [c for c in ['GDP_per_capita', 'ECI', 'Is_Dominant', 'Dominant_Count',
                                'Max_Reserve_Concentration', 'Resource_Diversity',
                                'PC1', 'PC2'] 
                   if c in self.df_final.columns]
        
        share_cols = self.prod_share_cols + self.res_share_cols
        cols = base_cols + optional + self.clustering_features + share_cols
        
        if 'Dominant_Resources' in self.df_final.columns:
            self.df_final['Dominant_Resources_List'] = self.df_final['Dominant_Resources'].apply(
                lambda x: ', '.join(x) if isinstance(x, list) else ''
            )
            cols.append('Dominant_Resources_List')
        
        self.df_final[[c for c in cols if c in self.df_final.columns]].to_csv(output_path, index=False)
    


In [124]:
def main(find_optimal_k=True):  # Default to False for quick runs
    """Run clustering with NR_final.csv dataset."""
    
    config = Config(
        resources_path="https://raw.githubusercontent.com/AyaanTigdikar/Capstone/7b651cc0ba93cc9af63ff3cefd255e75913edf4e/workingdata/NR_final.csv",
        master_data_path="https://raw.githubusercontent.com/AyaanTigdikar/Capstone/7b651cc0ba93cc9af63ff3cefd255e75913edf4e/workingdata/master_data_wide.csv",
        
        n_clusters=5,
        use_production_for_clustering=True,
        use_reserves_for_clustering=False,
        use_reserves_for_dominance=True,
        
        aggregate_minerals=True,
        normalize_by_gdp=True,
        include_economic_complexity=False,
        include_resource_rents=False,
        
        diversity_threshold_pct=1.0,
        
        year_min=1990,
        year_max=2024,
        master_year=2019
    )
    
    analysis = ResourceClusteringAnalysis(config)
    df = analysis.run(find_optimal_k=find_optimal_k)
    
    # Create visualizations
    figures = analysis.visualize()
    
    # Export results
    analysis.export_results("clustering_results.csv")
    
    # Show figures
    for name, fig in figures.items():
        fig.show()
        logger.info(f"✅ Displayed {name} visualization")
    
    return analysis

if __name__ == "__main__":
    analysis = main(find_optimal_k=True)

NATURAL RESOURCES CLUSTERING - PRODUCTION VALUES DATASET

[1] LOADING DATA
Loading resources...
  Mapping country names to ISO codes...
  Removed 1083 rows without country codes
  29,415 rows, 22 resources, 156 countries
Loading master data (wide format)...
  Year 2019: 251 countries
  GDP per capita: 191 countries
  Economic Complexity: 144 countries

[2] PROCESSING
    No price data available - using production VOLUME
    Aggregating minerals into one feature (keeping individual for hover)
    Production: 152 countries, 4 features for clustering
    (Individual minerals kept for hover text)
    Reserves (dominance only): 18 resources

[3] CALCULATING SHARES

[4] DOMINANCE FROM RESERVES
  Dominant countries (>15.0% reserves): 13

[5] CREATING CLUSTERING FEATURES
  Merging with master data...
  Resource data countries: 150
  Master data countries: 251
  Overlap: 149
  Matched 146 countries with GDP data
  Created 4 production VALUE/GDP features (aggregated)
  Resource Diversity: Requir

✅ Displayed map visualization


✅ Displayed profiles visualization


✅ Displayed validation visualization
